# Trabalho prático de dados, parte 2

Nesta etapa do trabalho prático, o grupo precisava aplicar algum algoritmo de aprendizagem
(regras de associação, regressão, aprendizado supervisionado ou aprendizado não-supervisionado)
para classificar ou agrupar os dados e, assim, tentar prever algum acontecimento desconhecido, com
foco em evasão, nota do ENEM e CRA.

## Imports básicos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


## Lendo dataset

In [ ]:
df = pd.read_csv("../Datasets/dadosFiltrados.csv")
df.info()

### Selecionando atributos significativos

Vamos primeiramente realizar uma plotagem dos atributos do datasets para ver se conseguimos encontrar alguma correlação pelos gráficos. Para isso, foram selecionadas algumas colunas que possivelmente apresentarão resultados significativos

In [ ]:
dfPlot = df[[
    "Ano_Nascimento" ,
    "Sexo" ,
    "Campus" ,
    "Curso_Identificador" ,
    "UF_Nascimento" ,
    "Codigo_Situacao_Aluno" ,
    "Modalidade_Inscrita" ,
    "ENEM" ,
    "Num_Reprovacoes" ,
    "Raca" ,
    "CRA" ,
    "Area" ,
    "Admissao_Ano",
    "Saida_Ano",
]]
# Colunas não usadas
# ["Identificador" , 
# "Curso" ,
# "UF_Nascimento" ,
# "Admissao" ,
# "Saida" ,
# "Situacao_Aluno" ,
# "Situacao_Aluno_Agrupada" ,
# "Admissao_Semestre",
# "Saida_Semestre"]

# Vamos converter os atributos desse dataset que são categoricos para variaveis discretas
colunas = dfPlot.select_dtypes(include=['object']).columns
for col in colunas:
    dfPlot[col] = dfPlot[col].astype('category')
for col in colunas:
    dfPlot[col] = dfPlot[col].cat.codes
dfPlot = dfPlot[dfPlot['Saida_Ano'] != 0]
dfPlot


In [ ]:
plt.figure(figsize=(17, 17))
sns.pairplot(dfPlot, corner=True)
plt.savefig('Imagens/pairplot.png', format='png', dpi=300)

Para uma visualização mais detalhada, é recomendável abrir a imagem separadamente em [Imagens/pairplot.png](Imagens/pairplot.png)

In [ ]:
# ["Identificador" , 
# "Ano_Nascimento" ,
# "Sexo" ,
# "Campus" ,
# "Curso_Identificador" ,
# "Curso" ,
# "UF_Nascimento" ,
# "Municipio_Nascimento" ,
# "Admissao" ,
# "Saida" ,
# "Codigo_Situacao_Aluno" ,
# "Situacao_Aluno" ,
# "Situacao_Aluno_Agrupada" ,
# "Modalidade_Inscrita" ,
# "ENEM" ,
# "Num_Reprovacoes" ,
# "Raca" ,
# "CRA" ,
# "Area" ,
# "Admissao_Ano",
# "Admissao_Semestre",
# "Saida_Ano",
# "Saida_Semestre"]

analises possiveis:
mulheres reprovam menos
reprovação por campus
enem vs reprovçoes
cra reprovaçoes

## Analise 1: Prever CRA 
Para essa analise teremos que identificar os principais fatores que influenciam o coeficiente de rendimento acumulado. Para isso levaremos em consideração somente a variaveis areas e não todos os cursos em busca de uma melhor eficiência computacional. 

## Filtragem do  dados

In [ ]:
dataCRA = df.copy()

### Criando variavel tempo de permanencia
Para uma analise mais efetiva iremos criar a coluna `Anos_Cursados` que irá indicar o quantos anos o aluno precisou para se formar.

In [ ]:
dataCRA ['Anos_Cursados']= dataCRA['Saida_Ano'] - dataCRA['Admissao_Ano']

### Limpando DataSet

In [ ]:
print(dataCRA.size)
dataCRA = dataCRA.dropna()
dataCRA.size

Sabemos que no Brasil o tempo mínimo para se formar em um curso de graduação no Brasil varia de acordo diretrizes do Ministério da Educação (MEC). Sendo o menor tempo possível o de  2 anos, que é definido pelo MEC para os cursos tecnólogo, logo consideraremos apenas esse tempo para cima para analise.

In [ ]:
dataCRA = dataCRA[dataCRA['Anos_Cursados'] > 1]
print(dataCRA.size)

### Selecionando atributos significativos

Optamos por fazer uma limpeza considerando apenas as colunas relevantes, que não existe uma representação mais clara da mesma atravês de outras colunas ou que não limitamos a utilização anteriormente.
Desconsiderando as colunas:
 - Identificador
 - Curso_Identificador
 - Curso
 - UF_Nascimento
 - Municipio_Nascimento
 - Admissao
 - Saida
 - Situacao_Aluno
 - Situacao_Aluno_Agrupada
 - Admissao_Ano
 - Admissao_Semestre
 - Saida_Ano
 - Saida_Semestre

Feito essa filtragem dos dados para analise da relação das colunas relevantes.

In [ ]:
dataCRA = dataCRA[
    ["Ano_Nascimento" ,
    "Sexo" ,
    "Campus" ,
    "Modalidade_Inscrita" ,
    "ENEM" ,
    "Num_Reprovacoes" ,
    "Raca" ,
    "CRA" ,
    "Anos_Cursados",
    "Codigo_Situacao_Aluno",
    "Area"]
]

dataCRA.head(3)

#### Analise do comportamento das variaveis 

Para podermos visualizar o comportamento teremos que fazer uma conversão inicial somente a titulo de visualização

In [ ]:
dfTeste = dataCRA.copy()

colunas = dfTeste.select_dtypes(include=['object']).columns
for col in colunas:
    dfTeste[col] = dfTeste[col].astype('category')
for col in colunas:
    dfTeste[col] = dfTeste[col].cat.codes
dfTeste.head(2)

#### Conferindo se há alguma relação de forte dependencia dentre as variaveis

In [ ]:
X = dfTeste.drop('CRA', axis=1).columns

correlation = dfTeste[X].corr()
correlation_percent = correlation * 100

# Criar a máscara para a parte superior da matriz
matrix = np.triu(correlation_percent)

plt.figure(figsize=(15,8))
sns.heatmap(correlation_percent, 
            xticklabels=correlation_percent.columns, 
            yticklabels=correlation_percent.columns, 
            annot=True, 
            fmt='.1f',  # Formatar os valores com 1 casa decimal
            center=0, 
            cmap='coolwarm', 
            mask=matrix,
            annot_kws={"size": 10})  # Ajustar o tamanho da anotação se necessário



plt.title('Matriz de Correlação (%)')
plt.show()


Observando o mapa de calor conseguimos ver que não há uma porcentagem de correlação significativa, com o maximo chegando somente a 20,5% de correlação, logo não precisaremos fazer uma limpeza nesse sentido.

## Convertendo as variaveis categoricas em numericas

In [ ]:
dataCRA = pd.get_dummies(dataCRA, columns=['Modalidade_Inscrita', 'Campus', 'Raca', 'Sexo',"Codigo_Situacao_Aluno", "Area"], drop_first=True, dtype=int)
dataCRA.head(5)

Função acima pega cada uma das colunas e converge cada possivel valor categorico em uma coluna de zeros e uns.

## Realizando a previsão com Random Florest

In [ ]:
X = dataCRA.drop('CRA', axis=1)
Y = dataCRA['CRA']
X.columns

In [ ]:
# from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
X = dataCRA.drop('CRA', axis=1)
Y = dataCRA['CRA']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
        X,
        Y,
        test_size=0.3,
        random_state=0
)

In [ ]:
# Criando o modelo de RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)

# Treinando o modelo
rf.fit(X_train, Y_train)

# Predição usando o modelo treinado
rf_Y_predicted = rf.predict(X_test)

### Observando as previsões feitas

In [ ]:
df_previsoes = pd.DataFrame(X_test)
df_previsoes['CRA_Real'] = Y_test
df_previsoes['CRA_Previsto'] = rf_Y_predicted
df_previsoes['Diferenca_CRA'] = abs(df_previsoes['CRA_Real'] - df_previsoes['CRA_Previsto'])

In [ ]:
df_previsoes

### Avaliando o desempenho

In [ ]:
# Avaliando o modelo
r2 = r2_score(Y_test, rf_Y_predicted)
print(f"R-squared: {r2} or {(r2*100)}%")

### Analise desempenho do modelo

Um valor de R-quadrado de 0,8205 significa que aproximadamente 82.05% do CRA é explicada pelo modelo de regressão. Esse valor indica que o modelo tem uma boa capacidade de previsão e ajusta bem aos dados.

### Analisando quais variaveis significativas

In [ ]:
importances = rf.feature_importances_
features = X.columns
importances_df = pd.DataFrame({'Feature': features, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print(importances_df)

Para melhorar sua eficiência e generalização iremos reduzir as variaveis para somente as quinze mais significativas.

In [ ]:
# Reduzindo para as 15 variaveis mais importantes
top_features = [
    'Num_Reprovacoes',
    'Codigo_Situacao_Aluno_C',
    'Anos_Cursados',
    'ENEM',
    'Ano_Nascimento',
    'Codigo_Situacao_Aluno_D',
    'Codigo_Situacao_Aluno_M',
    'Area_CIENCIAS EXATAS E DA TERRA',
    'Area_CIENCIAS AGRARIAS',
    'Sexo_M',
    'Raca_Informação não disponível',
    'Campus_CAV',
    'Modalidade_Inscrita_9.0',
    'Raca_Pardo(a)',
    'Modalidade_Inscrita_5.0'
]


In [ ]:
X_top_features = X[top_features]

# Dividindo novamente os dados
X_train_top, X_test_top, Y_train_top, Y_test_top = train_test_split(X_top_features, Y, test_size=0.2, random_state=42)

# Criando e treinando o modelo
rf_top = RandomForestRegressor(n_estimators=100, random_state=42)
rf_top.fit(X_train_top, Y_train_top)

# Predição usando o modelo treinado
rf_Y_predicted_top = rf_top.predict(X_test_top)

### Observando as previsões feitas

In [ ]:
df_previsoes = pd.DataFrame(X_test_top)
df_previsoes['CRA_Real'] = Y_test_top
df_previsoes['CRA_Previsto'] = rf_Y_predicted_top
df_previsoes['Diferenca_CRA'] = abs(df_previsoes['CRA_Real'] - df_previsoes['CRA_Previsto'])

In [ ]:
df_previsoes

In [ ]:
# Avaliação do modelo
r2_top = r2_score(Y_test_top, rf_Y_predicted_top)

print(f"R-squared: {r2_top} ou {(r2_top * 100):.2f}%")

### Analise desempenho do modelo

O algoritmo apresentou um tempo de execução reduzido de 10,4 segundos para 5,6 segundos, o que representa uma queda de 37,5% no tempo de processamento. Essa melhoria na eficiência foi acompanhada por uma pequena perda de desempenho, com o R-squared diminuindo apenas 1,68%. Esse resultado sugere que, ao selecionar as variáveis mais relevantes, o modelo se tornou significativamente mais rápido sem comprometer substancialmente sua capacidade de explicar a variabilidade dos dados. Em resumo, a otimização resultou em um modelo mais eficiente e ágil, mantendo uma performance bastante próxima da original e com uma maior generalização.